# Synchronize Object and Base Pose TF

This notebook synchronizes data from extracted bagfiles. Assuming the data directory containt are like so:
> ```
dataDir
    base.txt # index of pose of base by time
    object.txt # index of pose of object by time
```

The resulting output is like so:

> ```
dataDir
    base_sync.txt # index of pose of base by time
    object_sync.txt # index of pose of object by time
```

In [29]:
from decimal import Decimal
import numpy as np
import pandas as pd
import os

## Configure parameters

In [30]:
dataDir = '/home/ruffsl/Desktop/bag_final/tide_occ'
# sample_rate = 1 # output sample rate (every x image pair)
# frame_rate = 30 # input frame rate (fps)
# tolerance = 0.3 # 60% period tolerance of frame rate
# object_to_base_offset = 0 # in seconds

## Create paths

In [31]:
base_file = os.path.join(dataDir,'base.txt')
object_file = os.path.join(dataDir,'object.txt')

base_sync_file = os.path.join(dataDir,'base_sync.txt')
object_sync_file = os.path.join(dataDir,'object_sync.txt')

## Load pose info into dataframe

In [32]:
base_df = pd.read_csv(base_file, header=None, delim_whitespace=True, dtype={0:str})
base_df = base_df.set_index(0)

object_df = pd.read_csv(object_file, header=None, delim_whitespace=True, dtype={0:str})
object_df = object_df.set_index(0)

## Load and associate data

In [33]:
base_array = np.array(base_df.index, dtype=np.dtype(Decimal))
object_array = np.array(object_df.index, dtype=np.dtype(Decimal))

idx = np.searchsorted(base_array, object_array) - 1
mask = idx >= 0
df = pd.DataFrame({"base_array":base_array[idx][mask], "object_array":object_array[mask]})

## Loop over matches and format as string for index

In [34]:
base_poses = df['base_array'].astype(basestring).values
object_poses = df['object_array'].astype(basestring).values

## Save pose file as well

In [35]:
base_df2 = base_df[base_df.index.map(lambda x: x in base_poses)]
base_df2.to_csv(base_sync_file, header=None)

object_df2 = object_df[object_df.index.map(lambda x: x in object_poses)]
object_df2.to_csv(object_sync_file, header=None)